# Data Engineering

In [201]:
import numpy as np
import pandas as pd

calendar = pd.read_csv("../data/calendar.csv")
sales = pd.read_csv("../data/sales_train_validation.csv")
# sample_submission = pd.read_csv("../data/sample_submission.csv")
sell_prices = pd.read_csv("../data/sell_prices.csv")

In [202]:
calendar.set_index('d')
sell_prices.set_index(["store_id", "item_id"])

wm_yr_wk  sell_price
store_id item_id                            
CA_1     HOBBIES_1_001     11325        9.58
         HOBBIES_1_001     11326        9.58
         HOBBIES_1_001     11327        8.26
         HOBBIES_1_001     11328        8.26
         HOBBIES_1_001     11329        8.26
...                          ...         ...
WI_3     FOODS_3_827       11617        1.00
         FOODS_3_827       11618        1.00
         FOODS_3_827       11619        1.00
         FOODS_3_827       11620        1.00
         FOODS_3_827       11621        1.00

[6841121 rows x 2 columns]

In [208]:
items = sales["item_id"].unique()
a = []
for s in items:
    a.append(s)
    if s=="FOODS_3_827":
        break
    

print(len(a), set(items)-set(a))

3049 set()


In [3]:
# sample = sales.sample(frac=0.1, replace=False, random_state=1)
# len(sample)

In [4]:
# # This is disaster, Too slow

# from tqdm.notebook import tqdm

# with tqdm(total=len(sample)) as pbar:
#     dfarray = []
#     for index, row in sample.iterrows():

#         s = sell_prices
#         s = s[s["store_id"]==row["store_id"]]
#         s = s[s["item_id"]==row["item_id"]]

#         weekly_idx = 0
#         acc_weekly_sale = 0
#         prev_weekly_sale = 0
#         cc = 0
#         for i in range(1, 1914):
#             d = calendar[calendar["d"]=="d_{}".format(i)]

#             wm_yr_wk = int(d["wm_yr_wk"])
#             ss = s[s["wm_yr_wk"]==wm_yr_wk]

#             sell_price = None
#             if len(ss)!=0:
#                 sell_price = ss["sell_price"]

#             rolling_mean = None
#             if i > 3:
#                 rolling_mean = 0
#                 rolling_mean += row["d_{}".format(i)]
#                 rolling_mean += row["d_{}".format(i-1)]
#                 rolling_mean += row["d_{}".format(i-2)]
#                 rolling_mean /= 3
            
#             if wm_yr_wk == weekly_idx:
#                 acc_weekly_sale += row["d_{}".format(i)]
#                 cc += 1
#             else:
#                 weekly_idx = wm_yr_wk
#                 prev_weekly_sale = acc_weekly_sale
#                 acc_weekly_sale = row["d_{}".format(i)]
#                 cc = 1
                
#             dfarray.append({
#                 "item_id": row["item_id"],
#                 "dept_id": row["dept_id"],
#                 "cat_id": row["cat_id"],
#                 "store_id": row["store_id"],
#                 "state_id": row["state_id"],
#                 "day": i,
#                 "wm_yr_wk": d["wm_yr_wk"],
#                 "wday": d["wday"],
#                 "month": d["month"],
#                 "year": d["year"],
#                 "event_name_1": d["event_name_1"],
#                 "event_type_1": d["event_type_1"],
#                 "event_name_2": d["event_name_2"],
#                 "event_type_2": d["event_type_2"],
#                 "snap_CA": d["snap_CA"],
#                 "snap_TX": d["snap_TX"],
#                 "snap_WI": d["snap_WI"],
#                 "sell_price": sell_price,
                
#                 "sale": row["d_{}".format(i)],
#                 "shifted_sale1": row["d_{}".format(i-1)] if i > 1 else None,
#                 "shifted_sale2": row["d_{}".format(i-2)] if i > 2 else None,
#                 "shifted_sale3": row["d_{}".format(i-3)] if i > 3 else None,
#                 "shifted_sale4": row["d_{}".format(i-4)] if i > 4 else None,
#                 "shifted_sale5": row["d_{}".format(i-5)] if i > 5 else None,
                
#                 "rolling_mean": rolling_mean,
#                 "prev_weekly_sale": prev_weekly_sale,
#                 "acc_weekly_sale": acc_weekly_sale,
#                 "avg_sale_this_week": acc_weekly_sale*1.0/cc,
#                 "next_sale": row["d_{}".format(i+1)] if i < 1913 else None,
#             })
            
#         if index%200 == 0:
#             df = pd.DataFrame(dfarray)
#             df.to_csv("expand_sale_{}.csv".format(index))
            
#         pbar.update(1)
# #         break


In [5]:
# df = pd.DataFrame(dfarray)
# df.to_csv("expand_sale.csv")

## Privot data v2

I still cannot run this code on my machine (it ate all my RAM) so I decided to run on colab.
https://colab.research.google.com/drive/1CzOnHyEAUIX6OLJtXV09TOXXosXaieZa#scrollTo=GN6oYkAXEzix


In [6]:
# df = sales.melt(id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"], var_name="d", value_name="sale")
# itemIds = sales["item_id"].unique()

# from tqdm.notebook import tqdm

# with tqdm(total=len(itemIds)) as pbar:
#     for item in itemIds:
#         subDf = df[df["id"]==item]
#         p = pd.merge(subDf, calendar, on='d')
#         p2 = pd.merge(p, sell_prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")
#         p2.to_csv("dataframes/expand_df_{}.csv".format(item))

#         print(item, "DONE")
#         pbar.update(1)



# Loading new datasets


In [7]:
itemIds = sales["item_id"].unique()

In [15]:
itemIds = ["HOBBIES_1_{:03d}".format(i) for i in range(1, 51)]
filenames = ["expand_df_"+s for s in itemIds]

In [71]:
df = pd.read_csv("dataframes/{}.csv".format(filenames[0]))
df.sort_values(by=['store_id', "date"], inplace=True)
df.reset_index(inplace=True)

In [72]:
df.head()

,index,Unnamed: 0,id,item_id,dept_id,cat_id,store_id,state_id,d,sale,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,10,10,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,20,20,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,30,30,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,...,2,2011,NaN,NaN,NaN,NaN,1,1,0,NaN
4,40,40,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,...,2,2011,NaN,NaN,NaN,NaN,1,0,1,NaN


In [73]:
dfv2 = df.drop(["Unnamed: 0", "id", "item_id", "dept_id", "cat_id"], axis=1)
dfv2.head()

,index,store_id,state_id,d,sale,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,0,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,10,CA_1,CA,d_2,0,2011-01-30,11101,Sunday,2,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,20,CA_1,CA,d_3,0,2011-01-31,11101,Monday,3,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,30,CA_1,CA,d_4,0,2011-02-01,11101,Tuesday,4,2,2011,NaN,NaN,NaN,NaN,1,1,0,NaN
4,40,CA_1,CA,d_5,0,2011-02-02,11101,Wednesday,5,2,2011,NaN,NaN,NaN,NaN,1,0,1,NaN


# Feature Engineering

In [126]:
#                 "shifted_sale1": row["d_{}".format(i-1)] if i > 1 else None,
#                 "shifted_sale2": row["d_{}".format(i-2)] if i > 2 else None,
#                 "shifted_sale3": row["d_{}".format(i-3)] if i > 3 else None,
#                 "shifted_sale4": row["d_{}".format(i-4)] if i > 4 else None,
#                 "shifted_sale5": row["d_{}".format(i-5)] if i > 5 else None,
                
#                 "rolling_mean": rolling_mean,
#                 "prev_weekly_sale": prev_weekly_sale,
#                 "acc_weekly_sale": acc_weekly_sale,
#                 "avg_sale_this_week": acc_weekly_sale*1.0/cc,
#                 "next_sale": row["d_{}".format(i+1)] if i < 1913 else None,

import math

def get_features(df):
    
    for i in range(1, 6):
        df[f"shift_t{i}"] = df.groupby('store_id')["sale"].shift(i)
         
    df["rolling_mean"] = df["sale"].shift(1).rolling(3, min_periods=1).mean()
    
    df["rolling_decay_mean"] = df["shift_t1"].copy()
    for i in range(2, 4):
        df["rolling_decay_mean"] += math.pow(0.9, i-1) * df[f"shift_t{i}"]
    df["rolling_decay_mean"] = df["rolling_decay_mean"]/3.0
    
    weekly_sale = df.groupby(['store_id', "wm_yr_wk"])["sale"].sum().reset_index()
    weekly_sale["prev_weekly_sale"] = weekly_sale.groupby('store_id')["sale"].shift(1)
    
    weekly_sale.drop(["sale"], axis=1, inplace=True)
    df = pd.merge(df, weekly_sale, on=["store_id", "wm_yr_wk"], how="left")
    
    df["acc_sale_by_week"] = df.groupby(['store_id', "wm_yr_wk"])["sale"].cumsum()

    #skew, kurt??
    #is_weekend
    return df

def get_label(df):
    return df.groupby('store_id')["sale"].shift(-1)


dfv3 = get_features(dfv2)
next_sale = get_label(dfv3)

In [134]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
dfv3.columns
dfv3.head(1)

,index,store_id,state_id,d,sale,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,shift_t1,shift_t2,shift_t3,shift_t4,shift_t5,rolling_mean,rolling_decay_mean,prev_weekly_sale,acc_sale_by_week
0,0,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [193]:
X = dfv3.drop(["index", "d", "date", "wm_yr_wk", "weekday"], axis=1)
X["wday"] = X["wday"].astype('category')
X["month"] = X["month"].astype('category')
X["year"] = X["year"].astype('category')

# eventNames = X["event_name_1"].unique().tolist()+X["event_name_2"].unique().tolist()

# This prove that event1 unique values covers event2
a = set(X["event_name_1"].unique().tolist())
b = set(X["event_name_2"].unique().tolist())
print(b - a)

#TODO: deal with events
# X1 = pd.get_dummies(X, prefix=['event1'], columns = ['event_name_1'], dummy_na=True, drop_first=True)
# X2 = pd.get_dummies(X, prefix=['event2'], columns = ['event_name_2'], dummy_na=True, drop_first=True)

X["holiday"] = pd.notna(X["event_name_1"])
X = X.drop(["event_name_1", "event_name_2", "event_type_1", "event_type_2"], axis=1)

# Ignore where sell_price is nan which indicates that it did not sell in that day
X["pred"] = next_sale
X = X.dropna(subset=['sell_price', "pred"])

Y = X["pred"]
X = X.drop(["pred"], axis=1)
X.head()

set()


,store_id,state_id,sale,wday,month,year,snap_CA,snap_TX,snap_WI,sell_price,shift_t1,shift_t2,shift_t3,shift_t4,shift_t5,rolling_mean,rolling_decay_mean,prev_weekly_sale,acc_sale_by_week,holiday
896,CA_1,CA,0,1,7,2013,0,1,0,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False
897,CA_1,CA,0,2,7,2013,0,0,1,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False
898,CA_1,CA,0,3,7,2013,0,1,1,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False
899,CA_1,CA,0,4,7,2013,0,0,0,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False
900,CA_1,CA,0,5,7,2013,0,0,0,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False


In [194]:
from sklearn import preprocessing, metrics

cat = ['store_id', 'state_id']
for feature in cat:
    encoder = preprocessing.LabelEncoder()
    X[feature] = encoder.fit_transform(X[feature])

In [195]:
X["snap_CA"] = X["snap_CA"].astype('int')
X["snap_TX"] = X["snap_TX"].astype('int')
X["snap_WI"] = X["snap_WI"].astype('int')
X.head()

,store_id,state_id,sale,wday,month,year,snap_CA,snap_TX,snap_WI,sell_price,shift_t1,shift_t2,shift_t3,shift_t4,shift_t5,rolling_mean,rolling_decay_mean,prev_weekly_sale,acc_sale_by_week,holiday
896,0,0,0,1,7,2013,0,1,0,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False
897,0,0,0,2,7,2013,0,0,1,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False
898,0,0,0,3,7,2013,0,1,1,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False
899,0,0,0,4,7,2013,0,0,0,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False
900,0,0,0,5,7,2013,0,0,0,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False


# Train LBGM 

In [197]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    
params = {
#     'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'poisson',
    'n_jobs': -1,
    'seed': 20,
    'learning_rate': 0.1,
    'alpha': 0.1,
    'lambda': 0.1,
    'bagging_fraction': 0.66,
    'bagging_freq': 2, 
    'colsample_bytree': 0.77}

train_set = lgb.Dataset(x_train, y_train)
test_set = lgb.Dataset(x_test, y_test)
    
model = lgb.train(params, train_set, num_boost_round = 2000, early_stopping_rounds = 200, valid_sets = [train_set, test_set], verbose_eval = 100)
# joblib.dump(model, 'lgbm_0.sav')
    
val_pred = model.predict(x_test, num_iteration=model.best_iteration)
val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_test))
print(f'RMSE: {val_score}')


# # def predict(test, submission):
# #     predictions = test[['id', 'date', 'demand']]
# #     predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
# #     predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

# #     evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
# #     evaluation = submission[submission['id'].isin(evaluation_rows)]

# #     validation = submission[['id']].merge(predictions, on = 'id')
# #     final = pd.concat([validation, evaluation])
# #     final.to_csv('submission.csv', index = False)
    

C:\Users\Tk\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 0.635486	valid_1's rmse: 0.712139
[200]	training's rmse: 0.581872	valid_1's rmse: 0.716319
Early stopping, best iteration is:
[36]	training's rmse: 0.679134	valid_1's rmse: 0.706123
RMSE: 0.7061227848039181
